In [1]:
import redis
import yaml
import numpy as np
import random
import dask.array as da
from dask.diagnostics import ProgressBar

In [2]:
with open('../settings.yaml') as f:
    sets = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
r = redis.Redis(sets['redis']['host'], port=sets['redis']['port'], db=0)

In [4]:
#https://math.stackexchange.com/questions/2858815/understanding-formula-for-hyperplanes

In [5]:
H,W=100000,512
L = 4 # used to set the number of hyperplanes
HPs = pow(2,L)  # number of hyperplanes, equals to number of bits in key

In [6]:
pow(2,HPs)

65536

In [7]:
Y = da.random.normal(size=(H, W), chunks=(10000,W))

In [8]:
idxes = np.empty([HPs, 2], dtype=np.int)
for i in range(HPs):
    idxes[i,:]=np.random.choice(a=H, size=2, replace=False)

In [9]:
a=Y[idxes[:,0]]
b=Y[idxes[:,1]]

In [10]:
b_a=b-a
a_norm2=da.linalg.norm(a, axis=1)
b_norm2=da.linalg.norm(b, axis=1)
c=(b_norm2-a_norm2)/2

In [11]:
with ProgressBar():
    keys=da.ma.masked_greater(Y.dot(b_a.T)-c,0).compute()

[########################################] | 100% Completed |  2.6s


In [12]:
tmp=np.packbits(keys.mask, axis=1)
tmpv=np.array([v.tobytes() for v in tmp])
keys_bits=da.from_array(tmpv, chunks=(10000,))

In [13]:
#assert keys_bits.shape[1]*8==HPs
assert len(Y.chunks[0]) == len(keys_bits.chunks[0])

In [14]:
def vec2redis(ks,vs):
    for k,v in zip(np.array(ks),np.array(vs)):
        r.lpush(k.tobytes(), np.array(v).tobytes())
    return np.array([1,1])[None]
res=da.map_blocks(vec2redis,  keys_bits, Y, chunks=(1,1), dtype='float32')

In [15]:
with ProgressBar():
    rr=res.compute()

[########################################] | 100% Completed |  1min 55.0s
